# Calcul du nombre de prélèvements CVM non conforme par commune et par année

L'objectif de ce notebook est de partir de la liste de communes cog_communes, et pour chaque commune et chaque année, calculer le nombre de prélèvements non conformes pour le CVM.

Il y aura plusieurs aggrégations à faire :
- commune (inseecommune) peut avoir plusieurs UDIs (cdreseau)
- un prélèvement (referenceprel) peut être rattaché à plusieurs UDIs (cdreseau)
- un prélèvement (referenceprel) peut être composé de plusieurs paramètres (cdparametresiseeaux) ; mais dans le cas du CVM, il y a un seul paramètre selon la catégorisation de Pauline, donc c'est plus simple


La résultat final est dans la dernière cellule. Toutes les cellules précédentes sont là pour aider à la compréhension, en présentant des résultats intermédiaires.

In [1]:
%load_ext sql
%sql duckdb:///../../database/data.duckdb
%config SqlMagic.displaylimit = 10


The 'toml' package isn't installed. To load settings from pyproject.toml or ~/.jupysql/config, install with: pip install toml

Connecting to 'duckdb:///../../database/data.duckdb'

In [2]:
%sqlcmd tables

Name
edc_communes
edc_prelevements
edc_resultats
mapping_categories


In [3]:
%sql select * from read_csv("https://www.insee.fr/fr/statistiques/fichier/7766585/v_commune_2024.csv", header=true)

Running query in 'duckdb:///../../database/data.duckdb'

TYPECOM,COM,REG,DEP,CTCD,ARR,TNCC,NCC,NCCENR,LIBELLE,CAN,COMPARENT
COM,01001,84,01,01D,012,5,ABERGEMENT CLEMENCIAT,Abergement-Clémenciat,L'Abergement-Clémenciat,0108,None
COM,01002,84,01,01D,011,5,ABERGEMENT DE VAREY,Abergement-de-Varey,L'Abergement-de-Varey,0101,None
COM,01004,84,01,01D,011,1,AMBERIEU EN BUGEY,Ambérieu-en-Bugey,Ambérieu-en-Bugey,0101,None
COM,01005,84,01,01D,012,1,AMBERIEUX EN DOMBES,Ambérieux-en-Dombes,Ambérieux-en-Dombes,0122,None
COM,01006,84,01,01D,011,1,AMBLEON,Ambléon,Ambléon,0104,None
COM,01007,84,01,01D,011,1,AMBRONAY,Ambronay,Ambronay,0101,None
COM,01008,84,01,01D,011,1,AMBUTRIX,Ambutrix,Ambutrix,0101,None
COM,01009,84,01,01D,011,1,ANDERT ET CONDON,Andert-et-Condon,Andert-et-Condon,0104,None
COM,01010,84,01,01D,011,1,ANGLEFORT,Anglefort,Anglefort,0110,None
COM,01011,84,01,01D,014,1,APREMONT,Apremont,Apremont,0114,None


In [4]:
%sql select *  from edc_communes

Running query in 'duckdb:///../../database/data.duckdb'

inseecommune,nomcommune,quartier,cdreseau,nomreseau,debutalim,de_partition,de_ingestion_date,de_dataset_datetime
01001,ABERGEMENT-CLEMENCIAT (L'),-,001000556,BDS ST DIDIER/CHALARONNE,2010-09-07,2020,2025-02-14,20230811-150005
01002,ABERGEMENT-DE-VAREY (L'),-,001000369,L'ABERGEMENT-DE-VAREY,2010-09-07,2020,2025-02-14,20230811-150005
01004,AMBERIEU-EN-BUGEY,Vareilles,001000248,AMBERIEU VAREILLES,2010-09-07,2020,2025-02-14,20230811-150005
01004,AMBERIEU-EN-BUGEY,St Germain_Brédevent,001000249,AMBERIEU SAINT GERMAIN DOUVRES,2010-09-07,2020,2025-02-14,20230811-150005
01004,AMBERIEU-EN-BUGEY,ville,001000251,AMBERIEU VILLE,2010-09-07,2020,2025-02-14,20230811-150005
01005,AMBERIEUX-EN-DOMBES,Est,001000850,BDS CHATANIER,2023-01-01,2020,2025-02-14,20230811-150005
01005,AMBERIEUX-EN-DOMBES,Ouest,001000850,BDS CHATANIER,2021-08-01,2020,2025-02-14,20230811-150005
01006,AMBLEON,-,001000235,AMBLEON,2010-09-07,2020,2025-02-14,20230811-150005
01007,AMBRONAY,-,001000003,AMBRONAY,2010-09-07,2020,2025-02-14,20230811-150005
01008,AMBUTRIX,Ambutrix centre,001000254,AMBUTRIX MAIRIE,2010-09-07,2020,2025-02-14,20230811-150005


In [5]:
%sql select *  from edc_prelevements

Running query in 'duckdb:///../../database/data.duckdb'

cddept,cdreseau,inseecommuneprinc,nomcommuneprinc,cdreseauamont,nomreseauamont,pourcentdebit,referenceprel,dateprel,heureprel,conclusionprel,ugelib,distrlib,moalib,plvconformitebacterio,plvconformitechimique,plvconformitereferencebact,plvconformitereferencechim,de_partition,de_ingestion_date,de_dataset_datetime
001,001000003,01007,AMBRONAY,None,None,None,00100119766,2020-02-13,11h40,Eau d'alimentation conforme aux exigences de qualité en vigueur pour l'ensemble des paramètres mesurés.,SI REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,C,C,C,C,2020,2025-02-14,20230811-150005
001,001000003,01007,AMBRONAY,001001304,TTP (CLG) AMBRONAY,100 %,00100120290,2020-03-17,11h15,Eau d'alimentation conforme aux exigences de qualité en vigueur pour l'ensemble des paramètres mesurés.,SI REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,C,C,C,C,2020,2025-02-14,20230811-150005
001,001000003,01007,AMBRONAY,None,None,None,00100120717,2020-05-14,11h54,Eau d'alimentation conforme aux exigences de qualité en vigueur pour l'ensemble des paramètres mesurés.,SI REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,C,C,C,C,2020,2025-02-14,20230811-150005
001,001000003,01007,AMBRONAY,None,None,None,00100121072,2020-06-11,12h00,Eau d'alimentation conforme aux exigences de qualité en vigueur pour l'ensemble des paramètres mesurés.,SI REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,C,C,C,C,2020,2025-02-14,20230811-150005
001,001000003,01007,AMBRONAY,None,None,None,00100121547,2020-07-08,11h09,Eau d'alimentation conforme aux exigences de qualité en vigueur pour l'ensemble des paramètres mesurés.,SI REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,C,C,C,C,2020,2025-02-14,20230811-150005
001,001000003,01007,AMBRONAY,001001304,TTP (CLG) AMBRONAY,100 %,00100121589,2020-07-08,09h05,Eau d'alimentation conforme aux exigences de qualité en vigueur pour l'ensemble des paramètres mesurés.,SI REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,C,C,C,C,2020,2025-02-14,20230811-150005
001,001000003,01007,AMBRONAY,None,None,None,00100122035,2020-08-13,10h44,Eau d'alimentation conforme aux exigences de qualité en vigueur pour l'ensemble des paramètres mesurés.,SI REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,C,C,C,C,2020,2025-02-14,20230811-150005
001,001000003,01007,AMBRONAY,None,None,None,00100122549,2020-09-14,12h51,Eau d'alimentation conforme aux exigences de qualité en vigueur pour l'ensemble des paramètres mesurés.,SI REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,C,C,C,C,2020,2025-02-14,20230811-150005
001,001000003,01007,AMBRONAY,None,None,None,00100123096,2020-10-12,08h25,Eau d'alimentation conforme aux exigences de qualité en vigueur pour l'ensemble des paramètres mesurés.,SI REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,C,C,C,C,2020,2025-02-14,20230811-150005
001,001000003,01007,AMBRONAY,001001304,TTP (CLG) AMBRONAY,100 %,00100123173,2020-10-12,09h15,Eau d'alimentation conforme aux exigences de qualité en vigueur pour l'ensemble des paramètres mesurés.,SI REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,C,C,C,C,2020,2025-02-14,20230811-150005


In [6]:
%%sql
    select *
    from "edc_resultats"
    where cdparametresiseeaux = 'CLVYL' and valtraduite > 0.5

# on liste l'ensemble des analyses non conformes pour le paramètre CVM

Running query in 'duckdb:///../../database/data.duckdb'

cddept,referenceprel,cdparametresiseeaux,cdparametre,libmajparametre,libminparametre,libwebparametre,qualitparam,insituana,rqana,cdunitereferencesiseeaux,cdunitereference,limitequal,refqual,valtraduite,casparam,referenceanl,de_partition,de_ingestion_date,de_dataset_datetime
002,00200180193,CLVYL,1753,CHLORURE DE VINYL MONOMÈRE,Chlorure de vinyl monomère,None,N,L,"1,0",µg/L,133,<=0.5 µg/L,None,1.0,75-01-4,00200181303,2020,2025-02-14,20230811-150005
004,00400122403,CLVYL,1753,CHLORURE DE VINYL MONOMÈRE,Chlorure de vinyl monomère,None,N,L,"0,64",µg/L,133,<=0.5 µg/L,None,0.64,75-01-4,00400122467,2020,2025-02-14,20230811-150005
004,00400122776,CLVYL,1753,CHLORURE DE VINYL MONOMÈRE,Chlorure de vinyl monomère,None,N,L,"0,529",µg/L,133,<=0.5 µg/L,None,0.529,75-01-4,00400122840,2020,2025-02-14,20230811-150005
004,00400123465,CLVYL,1753,CHLORURE DE VINYL MONOMÈRE,Chlorure de vinyl monomère,None,N,L,"0,531",µg/L,133,<=0.5 µg/L,None,0.531,75-01-4,00400123529,2020,2025-02-14,20230811-150005
004,00400123960,CLVYL,1753,CHLORURE DE VINYL MONOMÈRE,Chlorure de vinyl monomère,None,N,L,"2,00",µg/L,133,<=0.5 µg/L,None,2.0,75-01-4,00400124024,2020,2025-02-14,20230811-150005
004,00400124119,CLVYL,1753,CHLORURE DE VINYL MONOMÈRE,Chlorure de vinyl monomère,None,N,L,"1,40",µg/L,133,<=0.5 µg/L,None,1.4,75-01-4,00400124183,2020,2025-02-14,20230811-150005
005,00500110055,CLVYL,1753,CHLORURE DE VINYL MONOMÈRE,Chlorure de vinyl monomère,None,N,L,"0,79",µg/L,133,<=0.5 µg/L,None,0.79,75-01-4,00500117171,2020,2025-02-14,20230811-150005
005,00500112175,CLVYL,1753,CHLORURE DE VINYL MONOMÈRE,Chlorure de vinyl monomère,None,N,L,"2,575",µg/L,133,<=0.5 µg/L,None,2.575,75-01-4,00500119367,2020,2025-02-14,20230811-150005
005,00500112309,CLVYL,1753,CHLORURE DE VINYL MONOMÈRE,Chlorure de vinyl monomère,None,N,L,"1,396",µg/L,133,<=0.5 µg/L,None,1.396,75-01-4,00500119512,2020,2025-02-14,20230811-150005
006,00600221382,CLVYL,1753,CHLORURE DE VINYL MONOMÈRE,Chlorure de vinyl monomère,None,N,L,"1,7",µg/L,133,<=0.5 µg/L,None,1.7,75-01-4,00600221398,2020,2025-02-14,20230811-150005


In [7]:
%%sql

with "resultats_cvm_non_conformes" as (
    select *
    from "edc_resultats"
    where cdparametresiseeaux = 'CLVYL' and valtraduite > 0.5
),
"prelevements_cvm_non_conformes_par_annee_et_cdreseau" as (
    select
        "cdreseau",
        "resultats_cvm_non_conformes"."de_partition",
        count(*) as "count_prelevements_non_conformes",
        string_agg("resultats_cvm_non_conformes"."referenceprel") as "list_referenceprels",
        string_agg("valtraduite") as "list_valtraduite"
    from "resultats_cvm_non_conformes"
    left join "edc_prelevements" on
        "edc_prelevements"."referenceprel" = "resultats_cvm_non_conformes"."referenceprel"
        and
        "edc_prelevements"."de_partition" = "resultats_cvm_non_conformes"."de_partition"
    group by "cdreseau", "resultats_cvm_non_conformes"."de_partition"
)
select * from "prelevements_cvm_non_conformes_par_annee_et_cdreseau"

Running query in 'duckdb:///../../database/data.duckdb'

cdreseau,de_partition,count_prelevements_non_conformes,list_referenceprels,list_valtraduite
087000053,2021,2,"08700111373,08700111374","0.88,0.87"
087001148,2021,1,08700111174,0.57
087001426,2021,2,"08700109684,08700110502","0.54,0.67"
087001433,2021,3,"08700108765,08700111067,08700109784","0.63,0.56,1.1"
087001541,2021,2,"08700109701,08700110621","2.6,3.4"
087003627,2021,2,"08700109959,08700111085","1.1,0.75"
032003922,2022,1,03200089345,6.6
053000149,2022,1,05300115885,0.97
087000403,2021,1,08700109786,0.52
087000420,2021,7,"08700107649,08700108671,08700108672,08700110240,08700110280,08700111053,08700109904","0.98,0.95,1.8,1.2,1.1,1.1,1.1"


In [8]:
%%sql

with "resultats_cvm_non_conformes" as (
    select *
    from "edc_resultats"
    where cdparametresiseeaux = 'CLVYL' and valtraduite > 0.5
),
"prelevements_cvm_non_conformes_par_annee_et_cdreseau" as (
    select
        "cdreseau",
        "resultats_cvm_non_conformes"."de_partition",
        count(*) as "count_prelevements_non_conformes",
        string_agg("resultats_cvm_non_conformes"."referenceprel") as "list_referenceprels",
        string_agg("valtraduite") as "list_valtraduite"
    from "resultats_cvm_non_conformes"
    left join "edc_prelevements" on
        "edc_prelevements"."referenceprel" = "resultats_cvm_non_conformes"."referenceprel"
        and
        "edc_prelevements"."de_partition" = "resultats_cvm_non_conformes"."de_partition"
    group by "cdreseau", "resultats_cvm_non_conformes"."de_partition"
),
"communes_count_prelevements_cvm_non_conformes_par_annee" as (
    select
        "inseecommune",
        "edc_communes"."de_partition",
        count(distinct "edc_communes"."cdreseau") as "nbr_cdreseau",
        coalesce(sum(count_prelevements_non_conformes), 0) as "count_prelevements_non_conformes"
    from "edc_communes"
    left join "prelevements_cvm_non_conformes_par_annee_et_cdreseau" on
        "prelevements_cvm_non_conformes_par_annee_et_cdreseau"."cdreseau" = "edc_communes"."cdreseau"
        and
        "prelevements_cvm_non_conformes_par_annee_et_cdreseau"."de_partition" = "edc_communes"."de_partition"
    group by "inseecommune", "edc_communes"."de_partition"
)
select * from communes_count_prelevements_cvm_non_conformes_par_annee


Running query in 'duckdb:///../../database/data.duckdb'

inseecommune,de_partition,nbr_cdreseau,count_prelevements_non_conformes
88495,2024,1,0
88497,2024,1,0
88505,2024,1,0
88509,2024,1,0
89002,2024,1,0
89005,2024,2,0
89009,2024,1,0
89014,2024,1,0
89018,2024,1,0
89023,2024,1,0


In [9]:
%%sql

with "resultats_cvm_non_conformes" as (
    select *
    from "edc_resultats"
    where cdparametresiseeaux = 'CLVYL' and valtraduite > 0.5
),
"prelevements_cvm_non_conformes_par_annee_et_cdreseau" as (
    select
        "cdreseau",
        "resultats_cvm_non_conformes"."de_partition",
        count(*) as "count_prelevements_non_conformes",
        string_agg("resultats_cvm_non_conformes"."referenceprel") as "list_referenceprels",
        string_agg("valtraduite") as "list_valtraduite"
    from "resultats_cvm_non_conformes"
    left join "edc_prelevements" on
        "edc_prelevements"."referenceprel" = "resultats_cvm_non_conformes"."referenceprel"
        and
        "edc_prelevements"."de_partition" = "resultats_cvm_non_conformes"."de_partition"
    group by "cdreseau", "resultats_cvm_non_conformes"."de_partition"
),
"communes_count_prelevements_cvm_non_conformes_par_annee" as (
    select
        "inseecommune",
        "edc_communes"."de_partition",
        count(distinct "edc_communes"."cdreseau") as "nbr_cdreseau",
        coalesce(sum(count_prelevements_non_conformes), 0) as "count_prelevements_non_conformes"
    from "edc_communes"
    left join "prelevements_cvm_non_conformes_par_annee_et_cdreseau" on
        "prelevements_cvm_non_conformes_par_annee_et_cdreseau"."cdreseau" = "edc_communes"."cdreseau"
        and
        "prelevements_cvm_non_conformes_par_annee_et_cdreseau"."de_partition" = "edc_communes"."de_partition"
    group by "inseecommune", "edc_communes"."de_partition"
),
"annees" as (SELECT unnest(generate_series(2020, 2024)) as "annee")
select
    "cog"."COM" as "commune_code_insee",
    "cog"."LIBELLE" as "commune_nom",
    a."annee",
    coalesce("nbr_cdreseau", 0) as "nbr_cdreseau",
    coalesce("count_prelevements_non_conformes", 0) as "count_prelevements_non_conformes"
from read_csv("https://www.insee.fr/fr/statistiques/fichier/7766585/v_commune_2024.csv", header=true) as "cog"
cross join 
    "annees" a
left join "communes_count_prelevements_cvm_non_conformes_par_annee" on
   "cog"."COM" = "communes_count_prelevements_cvm_non_conformes_par_annee"."inseecommune"
   and
   a."annee"::string =  "communes_count_prelevements_cvm_non_conformes_par_annee"."de_partition"

# pour tester une commune avec un prélèvement non conforme, ajouter :
# where "COM" = '12139'

Running query in 'duckdb:///../../database/data.duckdb'

commune_code_insee,commune_nom,annee,nbr_cdreseau,count_prelevements_non_conformes
01001,L'Abergement-Clémenciat,2020,1,0
01002,L'Abergement-de-Varey,2020,1,0
01004,Ambérieu-en-Bugey,2020,3,0
01005,Ambérieux-en-Dombes,2020,1,0
01006,Ambléon,2020,1,0
01007,Ambronay,2020,1,0
01008,Ambutrix,2020,2,0
01009,Andert-et-Condon,2020,1,0
01010,Anglefort,2020,2,0
01011,Apremont,2020,1,0
